## SITCOM Laser Tracker Analysis: Laser - Hexapod Coordinate Sensitivity

Here we study how the measured offsets change as we change the hexapod movements, generating a "sensitivity matrix". These plots are useful for determining the pivot point and the coordinate transformation matrix.

Relevant imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time, TimeDelta
import yaml
from numpy.polynomial import Polynomial

from lsst_efd_client import EfdClient
from lsst.sitcom.vandv.lasertracker import utils

%matplotlib inline
%reload_ext lab_black
%reload_ext autoreload
%autoreload 2

## Get data from EFD for two different dates

In [ ]:
start = Time("2024-03-28 06:59:00Z", scale="utc")
end = Time("2024-03-28 08:35:00Z", scale="utc")

client = EfdClient("usdf_efd")
fields = ["dX", "dY", "dZ", "dRX", "dRY", "target"]

offsets_mar28 = await client.select_time_series(
    "lsst.sal.LaserTracker.logevent_offsetsPublish",
    fields,
    start,
    end,
)

start = Time("2024-03-29 21:40:00Z", scale="utc")
end = Time("2024-03-29 23:29:00Z", scale="utc")

client = EfdClient("usdf_efd")
fields = ["dX", "dY", "dZ", "dRX", "dRY", "target"]

offsets_mar29 = await client.select_time_series(
    "lsst.sal.LaserTracker.logevent_offsetsPublish",
    fields,
    start,
    end,
)

# Extract offsets, target elevation, azimuth and rotator angle by component
# Remove first datapoint because it's the initial measurement before any offset is applied.
data_cam_mar28 = utils.extract_component_data(offsets_mar28, "Camera")[1:]
data_m2_mar28 = utils.extract_component_data(offsets_mar28, "M2")[1:]

data_cam_mar29 = utils.extract_component_data(offsets_mar29, "Camera")[1:]
data_m2_mar29 = utils.extract_component_data(offsets_mar29, "M2")[1:]

## Plot sensitivity matrix

### Camera

In [ ]:
# Set indices for each of the sensitivity matrix sequence.
sequence_init = [0, 5, 10, 15, 21]
sequence_end = [5, 10, 15, 20, 26]

# Set labels
sequence_label = ['dY', 'dX', 'dZ', 'dRX', 'dRY']
sequence_unit = ['(um)', '(um)', '(um)', '(deg)', '(deg)']

for idy in range(len(sequence_init)):
    plt.figure(figsize = (10, 6))
    for idx, field in enumerate(fields[:-1]):
        plt.subplot(2,3,idx + 1)

        # Plot measured offsets against input hexapod motion
        if idy < 3:
            x_values = np.linspace(-500, 500, 5)
        else:
            x_values = np.linspace(-0.15,0.15,5)
        plt.plot(x_values, data_cam_mar28[field][sequence_init[idy]:sequence_end[idy]] - np.mean(data_cam_mar28[field][sequence_init[idy]:sequence_end[idy]]), '.-', label = 'Mar 28')
        plt.plot(x_values, data_cam_mar29[field][sequence_init[idy]:sequence_end[idy]] - np.mean(data_cam_mar29[field][sequence_init[idy]:sequence_end[idy]]), '.-', label = 'Mar 29')

        # Make fit for the measured offsets
        new_poly, [residual, _, _, _] = Polynomial.fit(x_values, data_cam_mar29[field][sequence_init[idy]:sequence_end[idy]] - np.mean(data_cam_mar29[field][sequence_init[idy]:sequence_end[idy]]), 1, full=True)
        coefs_fit = np.flip(new_poly.convert().coef)
        y_fit = np.polyval(coefs_fit, x_values)
        plt.plot(x_values, y_fit, '.--k')

        # Plot slope of the fit
        coef_text = f'm = {coefs_fit[0]:.2e}'
        # Assuming the placement at the top right corner of the plot
        plt.text(0.95, 0.95, coef_text, 
                         horizontalalignment='right', 
                         verticalalignment='top', 
                         transform=plt.gca().transAxes, 
                         bbox=dict(facecolor='white', edgecolor='black', pad=3.0))

        # Format plot
        plt.xlabel(f'{sequence_label[idy]} {sequence_unit[idy]}')
        plt.xticks(x_values)
        if idx >= 3:
            plt.ylabel('Measured (deg)')
        else:
            plt.ylabel('Measured (um)')
        plt.title(f'{field} Offsets')
    
    plt.suptitle(f'Camera {sequence_label[idy]} sensitvity', fontsize = 14, fontweight = 'bold')
    plt.tight_layout()
    #plt.savefig(f'Camera_{sequence_label[idy]}_sensitivity.png', dpi = 300)

### M2

In [ ]:
# Set indices for each of the sensitivity matrix sequence.
sequence_init = [0, 5, 10, 15]
sequence_end = [5, 10, 15, 18]

for idy in range(len(sequence_init)):
    plt.figure(figsize=(10, 6))
    for idx, field in enumerate(fields[:-1]):
        plt.subplot(2, 3, idx + 1)

        # Plot measured offsets against input hexapod motion
        if idy < 3:
            x_values = np.linspace(-500, 500, 5)
        else:
            x_values = np.linspace(-0.15, 0.15, 3)
        plt.plot(
            x_values,
            data_m2_mar28[field][sequence_init[idy] : sequence_end[idy]]
            - np.mean(data_m2_mar28[field][sequence_init[idy] : sequence_end[idy]]),
            ".-",
            label="Mar 28",
        )
        plt.plot(
            x_values,
            data_m2_mar29[field][sequence_init[idy] : sequence_end[idy]]
            - np.mean(data_m2_mar29[field][sequence_init[idy] : sequence_end[idy]]),
            ".-",
            label="Mar 29",
        )

        # Make fit for the measured offsets
        new_poly, [residual, _, _, _] = Polynomial.fit(
            x_values,
            data_m2_mar29[field][sequence_init[idy] : sequence_end[idy]]
            - np.mean(data_m2_mar29[field][sequence_init[idy] : sequence_end[idy]]),
            1,
            full=True,
        )
        coefs_fit = np.flip(new_poly.convert().coef)
        y_fit = np.polyval(coefs_fit, x_values)
        plt.plot(x_values, y_fit, ".--k")

        # Plot slope of the fit
        coef_text = f"m = {coefs_fit[0]:.2e}"
        # Assuming the placement at the top right corner of the plot
        plt.text(
            0.95,
            0.95,
            coef_text,
            horizontalalignment="right",
            verticalalignment="top",
            transform=plt.gca().transAxes,
            bbox=dict(facecolor="white", edgecolor="black", pad=3.0),
        )

        # Format plot
        plt.xlabel(f"{sequence_label[idy]} {sequence_unit[idy]}")
        plt.xticks(x_values)
        if idx >= 3:
            plt.ylabel("Measured (deg)")
        else:
            plt.ylabel("Measured (um)")
        plt.title(f"{field} Offsets")
        plt.legend()

    plt.suptitle(f"M2 {sequence_label[idy]} sensitvity", fontsize=14, fontweight="bold")
    plt.tight_layout()
    # plt.savefig(f'M2_{sequence_label[idy]}_sensitivity.png', dpi = 300)